In [101]:
import vk
import pandas as pd
import time

In [102]:
# opennig vk session with access token.
session = vk.Session(access_token='54d58d4edda257bd4aaec6527ae8183c2967a42a157568dc4ffcf966151991eaf26237b2de81400b4a7be')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids=245725902 )

[{'first_name': 'Levon', 'last_name': 'Yeghiazaryan', 'uid': 245725902}]

In [103]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [104]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 178 ms


In [105]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [106]:
df.head(2)

,attachment,attachments,comments,date,from_id,id,likes,marked_as_ads,owner_id,post_source,post_type,reposts,text
0,"{'type': 'photo', 'photo': {'pid': 456239772, ...","[{'type': 'photo', 'photo': {'pid': 456239772,...","{'count': 0, 'can_post': 1}",1512322597,-70740559,1034,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",0,-70740559,"{'type': 'api', 'platform': 'iphone'}",post,"{'count': 0, 'user_reposted': 0}",Наиболее частым заболеванием хрусталика являет...


In [107]:
df.columns

Index(['attachment', 'attachments', 'comments', 'date', 'from_id', 'id',
       'likes', 'marked_as_ads', 'owner_id', 'post_source', 'post_type',
       'reposts', 'text'],
      dtype='object')

In [108]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

,deactivated,first_name,last_name,uid
0,deleted,Elena,Zatylkina,70740559


In [131]:
%%time
desired_time = 86400*24 # how much back we want to go in seconds
date = time.time() # now
excess_time = True
all_df = pd.DataFrame()
c=1
while excess_time:
    print(c) # print iteration count
    g=api.newsfeed.search(q=['глауком глаз'],count=200, end_time = date-1, start_time=1)
    if len(g)>1: # check if g contains data
        all_df = pd.concat([all_df,pd.DataFrame(g[1:])])
        date=g[-1]['date']
    else:
        time.sleep(0.3)
    c+=1
    if date<time.time()-desired_time: # check termination condition
        excess_time = False

        # resetting the index it make cicles of 0-200
all_df.reset_index(inplace=True)

# set all negative ids to positive
all_df.owner_id=abs(all_df.owner_id) 

 # get all users
all_id=pd.DataFrame(api.users.get(user_ids=abs(all_df['owner_id'])))

# fill active users status with valid
all_id.fillna('Valid',inplace=True) 

# select all deactivated user ids
deactivated_ids = list(all_id[all_id['deactivated']!='Valid'].uid)

# select all active user posts
active_users = all_df[-all_df.owner_id.isin(deactivated_ids)] 

1
2
Wall time: 6.55 s


In [133]:
active_users

,index,attachment,attachments,comments,copy_owner_id,copy_post_date,copy_post_id,copy_post_type,copy_text,date,...,id,likes,marked_as_ads,owner_id,post_id,post_source,post_type,reposts,signer_id,text
0,0,"{'type': 'photo', 'photo': {'pid': 456239201, ...","[{'type': 'photo', 'photo': {'pid': 456239201,...","{'count': 0, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512652132,...,473,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",0.0,75933684,NaN,"{'type': 'api', 'platform': 'android'}",post,"{'count': 0, 'user_reposted': 0}",NaN,ЛЕЧЕБНЫЙ ЧАЙ НА ОСНОВЕ КЛИТОРИИ<br>(синий чай)...
2,2,"{'type': 'photo', 'photo': {'pid': 456239766, ...","[{'type': 'photo', 'photo': {'pid': 456239766,...","{'count': 0, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512649518,...,779,"{'count': 1, 'user_likes': 0, 'can_like': 1, '...",0.0,141651923,NaN,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",NaN,Тихий убийца Е621- глутамат натрия <br> <br>Гл...
3,3,"{'type': 'photo', 'photo': {'pid': 456239143, ...","[{'type': 'photo', 'photo': {'pid': 456239143,...","{'count': 0, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512644765,...,27,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",0.0,158078511,NaN,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",NaN,♻ЛЕГЕНДАРНЫЙ УЛЬТРА-МАСКИРУЮЩИЙ ТОНАЛЬНЫЙ КРЕМ...
4,4,"{'type': 'album', 'album': {'aid': '238997047'...","[{'type': 'album', 'album': {'aid': '238997047...","{'count': 0, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512644104,...,5197,"{'count': 6, 'user_likes': 0, 'can_like': 1, '...",0.0,39839426,NaN,{'type': 'vk'},post,"{'count': 4, 'user_reposted': 0}",NaN,ВЕЧНОЗЕЛЕНАЯ ТЕМА <br> <br>ПОГОВОРИМ О МНОГООБ...
5,5,"{'type': 'photo', 'photo': {'pid': 456244154, ...","[{'type': 'photo', 'photo': {'pid': 456244154,...","{'count': 1, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512640895,...,5602,"{'count': 4, 'user_likes': 0, 'can_like': 1, '...",0.0,55397716,NaN,{'type': 'vk'},post,"{'count': 2, 'user_reposted': 0}",NaN,Глазные капли Уджала для лечения катаракты / U...
7,7,"{'type': 'photo', 'photo': {'pid': 456239534, ...","[{'type': 'photo', 'photo': {'pid': 456239534,...","{'count': 0, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512631462,...,418,"{'count': 4, 'user_likes': 0, 'can_like': 1, '...",0.0,133205533,NaN,{'type': 'mvk'},post,"{'count': 2, 'user_reposted': 0}",NaN,Риск глаукомы можно снизить на 73% за счет физ...
8,8,"{'type': 'photo', 'photo': {'pid': 456239493, ...","[{'type': 'photo', 'photo': {'pid': 456239493,...","{'count': 1, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512626353,...,3539,"{'count': 69, 'user_likes': 0, 'can_like': 1, ...",0.0,44013675,NaN,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",NaN,Риск глаукомы можно снизить на 73% за счет физ...
13,13,NaN,NaN,"{'count': 0, 'can_post': 1}",NaN,NaN,NaN,NaN,NaN,1512560010,...,511743,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",0.0,125962295,511449.0,{'type': 'vk'},reply,"{'count': 0, 'user_reposted': 0}",NaN,"[id208237910|Лена], глаукома сопровождается фа..."
15,15,"{'type': 'photo', 'photo': {'pid': 456245959, ...","[{'type': 'photo', 'photo': {'pid': 456245959,...","{'count': 0, 'can_post': 0}",NaN,NaN,NaN,NaN,NaN,1512551520,...,10918,"{'count': 4, 'user_likes': 0, 'can_like': 1, '...",0.0,36241502,NaN,{'type': 'vk'},post,"{'count': 1, 'user_reposted': 0}",NaN,Когда ты мама маленького сына — ты вообще не д...
16,16,"{'type': 'photo', 'photo': {'pid': 423740997, ...","[{'type': 'photo', 'photo': {'pid': 423740997,...","{'count': 0, 'groups_can_post': True, 'can_pos...",NaN,NaN,NaN,NaN,NaN,1512547061,...,3057,"{'count': 2, 'user_likes': 0, 'can_like': 1, '...",NaN,210616375,NaN,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",NaN,"ВНИМАНИЕ, МЕДИЦИНСКИЙ ЦЕНТР СЕМЕЙНЫЙ ДОКТОР 👍👍..."


In [121]:
active_users.shape

(141, 21)